This jupyter notebook contains various codes which the team members used for scraping data or downloading existing dataset.

* same code might have been used for various websites


In [ ]:
#this is a .js file used for collecting bad words

const wash = require('washyourmouthoutwithsoap');
const fs = require('fs');

// Get the bad words for Malayalam
const badWordsMalayalam = wash.words('ml');

// Write the list to a file (JSON format)
fs.writeFileSync('bad_words_ml.json', JSON.stringify(badWordsMalayalam, null, 2));
console.log("Malayalam bad words list saved to bad_words_ml.json");

Various codes used for downloading the existing datasets::

In [ ]:
# Code 1

pip install huggingface_hub
pip install ipywidgets
from huggingface_hub import login
login()
pip install datasets
from datasets import load_dataset

ds = load_dataset("uonlp/CulturaX", "ml")
train_dataset = ds['train']
df = pd.DataFrame(train_dataset)

# Display the first few rows
print(df.head())
# Get the unique values from the 'source' column
unique_sources = df['source'].unique()

# Display the unique sources
print(unique_sources)
# Calculate the memory usage of the DataFrame
memory_usage_bytes = df.memory_usage(deep=True).sum()

# Convert bytes to gigabytes
memory_usage_gb = memory_usage_bytes / (1024 ** 3)

# Display the size of the dataset in GB
print(f"Dataset size: {memory_usage_gb:.2f} GB")
import os
source = "/Users/heerkubadia/Desktop/Sem - 5/Natural Language Processing/uonlp_CulturaX"
import os
import re

# Create the directory if it doesn't exist
os.makedirs("source", exist_ok=True)

# Function to clean up source names for valid file names
def clean_filename(source_name):
    # Remove invalid characters from the source name
    return re.sub(r'[^\w\-_. ]', '_', str(source_name))

# Iterate over each row of the DataFrame
for i, row in df.iterrows():
    # Get the cleaned source name
    source_name = clean_filename(row['source'])

    # Create a file name using the source name and index to avoid overwrites
    file_name = f"source/{source_name}_{i}.txt"

    # Write the URL and text to the file
    with open(file_name, "w") as f:
        f.write(f"{row['url']}\n")  # Write URL on the first line
        f.write(f"{row['text']}\n")  # Write text on the next lines

print("Files saved successfully.")
import json

# Open the jsonl file
with open('/Users/heerkubadia/Downloads/ml_clean_0001.jsonl', 'r') as f:
    # Read each line, parse it as JSON and store it in a list
    data = [json.loads(line) for line in f]

# Print the first item to see the structure
print(data[0])

import json

input_file = '/Users/heerkubadia/Downloads/ml_clean_0000.jsonl'  # Path to your .jsonl file
output_file = '/Users/heerkubadia/Desktop/Sem - 5/Natural Language Processing/MADLAD-400/ml_clean_0000.txt'  # Path to the text file where you want to append

with open(input_file, 'r') as f_in, open(output_file, 'a') as f_out:
    for line in f_in:
        # Parse each line as JSON
        data = json.loads(line)

        # Extract the 'text' field and clean it
        text = data.get('text', '').replace('\\n', '\n')

        # Append the cleaned text to the output file
        f_out.write(text + '\n')  # Add a newline between each entry
from datasets import load_dataset

# Load the Malayalam data from the 'verified' configuration
dataset = load_dataset("ai4bharat/sangraha", "verified", split="malayalam")

# Save each data entry to a separate text file
for idx, data in enumerate(dataset):
    with open(f"ai4bharat_{idx}.txt", "w") as f:
        f.write(data['text'])


In [ ]:
# Code 2

!pip install datasets
from datasets import load_dataset

# Load the dataset for Malayalam text
dataset = load_dataset("statmt/cc100", "ml")

# Combine all splits into a single dataset
all_texts = dataset['train']  # or dataset['test'] and concatenate if there are multiple splits

# Optionally save the combined dataset
all_texts.save_to_disk('./malayalam_cc100_combined_malayalam')


Python scripts for scraping (modified according to the website)

In [ ]:
# script 1:

import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, urlparse
import re

# checking if the text contains Malayalam characters
def contains_malayalam(text):
    # Unicode range for Malayalam characters
    malayalam_range = re.compile(r'[\u0D00-\u0D7F]')
    return bool(malayalam_range.search(text))

def fetch_malayalam_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # extracting text from current page
        text_content = soup.get_text()

        # filtering
        if contains_malayalam(text_content):
            return text_content
        return None
    except Exception as e:
        print(f"Failed to fetch {url}: {e}")
        return None

def save_to_file(content, filename, directory):
    if content:
        filepath = os.path.join(directory, filename)
        with open(filepath, 'w', encoding='utf-8') as file:
            file.write(content)
        print(f"Saved content to {filepath}")

def scrape_deshabhimani(base_url, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    to_visit = [base_url]
    visited = set()

    while to_visit:
        current_url = to_visit.pop(0)
        if current_url in visited:
            continue

        visited.add(current_url)
        print(f"Visiting {current_url}")

        content = fetch_malayalam_content(current_url)
        if content:
            filename = f"{len(visited)}.txt"
            save_to_file(content, filename, output_dir)

        # extracting hyperlinks
        try:
            response = requests.get(current_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            for link in soup.find_all('a', href=True):
                full_url = urljoin(base_url, link['href'])
                if is_deshabhimani_url(full_url, base_url) and full_url not in visited:
                    to_visit.append(full_url)
        except Exception as e:
            print(f"Failed to process {current_url}: {e}")

def is_deshabhimani_url(url, base_url):
    parsed_url = urlparse(url)
    return parsed_url.netloc == urlparse(base_url).netloc


base_url = 'https://www.deshabhimani.com/'
output_dir = r"C:\Users\jiyad\OneDrive\Desktop\iitgn random\NLP\deshabhimani_temp"

# calling the function
scrape_deshabhimani(base_url, output_dir)

In [ ]:
# script 2:

import os
import requests
import time

# Base raw URL for the files in the repository
raw_base_url = 'https://raw.githubusercontent.com/smc/corpus/master/text/ml-wiki/'

# Subfolders to traverse and their file ranges
subfolders = {
    'AD': 99,  # 99 files (wiki_00 to wiki_98)
    'AE': 18   # 18 files (wiki_00 to wiki_17)
}

# Create a folder to store the downloaded files
if not os.path.exists('ml-wiki'):
    os.makedirs('ml-wiki')

# Function to download files from a given subfolder
def download_files_from_subfolder(subfolder, file_range):
    # Raw URL for the subfolder
    subfolder_url = raw_base_url + subfolder + '/'

    # Generate file names (wiki_00 to wiki_98 or wiki_00 to wiki_17)
    file_names = [f'wiki_{str(i).zfill(2)}' for i in range(file_range)]

    for file_name in file_names:
        raw_file_url = subfolder_url + file_name
        print(f'Trying to download: {raw_file_url}')

        # Implementing retries with exponential backoff
        retries = 5
        for i in range(retries):
            try:
                # Download the file content
                response = requests.get(raw_file_url, timeout=10)

                # If request is successful, break out of the retry loop
                if response.status_code == 200:
                    # Create the subfolder locally if it doesn't exist
                    local_subfolder_path = os.path.join('ml-wiki', subfolder)
                    if not os.path.exists(local_subfolder_path):
                        os.makedirs(local_subfolder_path)

                    # Save the file locally
                    local_file_path = os.path.join(local_subfolder_path, file_name + '.txt')
                    with open(local_file_path, 'w', encoding='utf-8') as file:
                        file.write(response.text)

                    print(f'Successfully downloaded {file_name} from {subfolder}')
                    break
                else:
                    print(f'Failed to download {file_name} from {subfolder}, status code: {response.status_code}')
                    break

            except (requests.ConnectionError, requests.Timeout) as e:
                print(f"Error downloading {file_name}: {e}. Retrying {i + 1}/{retries}...")
                time.sleep(2 ** i)  # Exponential backoff
            except Exception as e:
                print(f"Unhandled exception for {file_name}: {e}")
                break

# Loop through the subfolders and download files from each
for subfolder, file_range in subfolders.items():
    download_files_from_subfolder(subfolder, file_range)

In [ ]:
# script 3:

import requests

# URL of the plain text file
url = 'https://raw.githubusercontent.com/smc/corpus/master/text/CMOKerala/CMOKerala.txt'

# Send a GET request to fetch the content
response = requests.get(url)

# Save the content to a text file
with open('CMOKerala.txt', 'w', encoding='utf-8') as file:
    file.write(response.text)

print("Content successfully saved to 'CMOKerala.txt'.")

In [ ]:
# script 4

import requests
from bs4 import BeautifulSoup
import csv
import os
import time

def fetch_links(page_number):
    url = f"https://news12.in/homepage-newspaper/page/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('a', href=True)

    links = []
    for article in articles:
        link = article['href']
        if link.startswith('/'):
            link = f"https://news12.in/{link}"
        links.append(link)

    return links

def save_links_to_csv(folder_path, page_number, links):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    filename = os.path.join(folder_path, f"{page_number}.csv")
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Link"])
        for link in links:
            writer.writerow([link])

if __name__ == "__main__":
    folder_path = '/mnt/HDFS1/language_nlp/malayalam_nlp_shristi/codes/news12/csv_folder'
    total_pages = 103
    for page_number in range(1, total_pages + 1):
        try:
            links = fetch_links(page_number)
            save_links_to_csv(folder_path, page_number, links)
            print(f"Page {page_number} processed successfully.")
        except Exception as e:
            print(f"An error occurred on page {page_number}: {e}")
            continue

In [ ]:
import os
import csv
import requests
from bs4 import BeautifulSoup
import re
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

def crawl_data_from_link_with_retry(link, max_retries=3, retry_interval=5):
    retries = 0
    while retries < max_retries:
        try:
            print(f"Processing link: {link}")
            response = requests.get(link)
            if response.status_code == 200:
                return response.content
            else:
                print(f"Failed to fetch data from {link}. Status code: {response.status_code}. Retrying...")
                retries += 1
                time.sleep(retry_interval)
        except Exception as e:
            print(f"An error occurred while fetching data from {link}: {e}. Retrying...")
            retries += 1
            time.sleep(retry_interval)
    print(f"Failed to fetch data from {link} after {max_retries} retries.")
    return None

def is_malayalamdu(text):
    malayalam_range = r'[\u0D00-\u0D7F]+'
    return re.match(malayalam_range, text)

def extract_data_from_html(html_content):
    extracted_data = []
    try:
        soup = BeautifulSoup(html_content, "html.parser")
        p_elements = soup.find_all('p')
        for p in p_elements:
            p_text = p.get_text(separator=' ').strip()
            if is_malayalam(p_text):
                extracted_data.append(p_text)
    except Exception as e:
        print(f"An error occurred while extracting malayalam data: {e}")
    return extracted_data

def process_link_and_write(link, text_file_path):
    html_content = crawl_data_from_link_with_retry(link)
    if html_content:
        extracted_data = extract_data_from_html(html_content)
        if extracted_data:
            try:
                with open(text_file_path, 'a', encoding='utf-8') as text_file:
                    cleaned_data = re.sub(r'\n\s*\n', '\n', '\n'.join(extracted_data)).strip()
                    text_file.write(cleaned_data + '\n\n')
                return True
            except Exception as e:
                print(f"An error occurred while writing to file {text_file_path}: {e}")
    return False

def process_csv_folder(csv_folder_path, corpus_directory, failed_csv_path):
    if not os.path.exists(corpus_directory):
        os.makedirs(corpus_directory)

    csv_files = [f for f in os.listdir(csv_folder_path) if f.endswith('.csv')]
    csv_files.sort(key=lambda x: int(os.path.splitext(x)[0]))

    with open(failed_csv_path, 'w', newline='', encoding='utf-8') as failed_csv:
        fieldnames = ['file_name', 'failed_link']
        writer = csv.DictWriter(failed_csv, fieldnames=fieldnames)
        writer.writeheader()

        for csv_file_name in csv_files:
            csv_file_path = os.path.join(csv_folder_path, csv_file_name)
            text_file_path = os.path.join(corpus_directory, f"{os.path.splitext(csv_file_name)[0]}.txt")

            with open(csv_file_path, 'r', encoding='ISO-8859-1') as csv_file:
                csv_reader = csv.reader(csv_file)
                links = [row[0] for row in csv_reader]

            total_links = len(links) - 1
            print(f"Processing {csv_file_name} with {total_links} links...")

            failed_links = []
            processed_links = 0

            with ThreadPoolExecutor(max_workers=10) as executor:
                future_to_link = {executor.submit(process_link_and_write, link, text_file_path): link for link in links[1:]}
                for future in as_completed(future_to_link):
                    link = future_to_link[future]
                    try:
                        success = future.result()
                        processed_links += 1
                        print(f"Processed {processed_links}/{total_links} links.")
                        if not success:
                            failed_links.append(link)
                    except Exception as e:
                        print(f"An error occurred while processing link {link}: {e}")
                        failed_links.append(link)

            if failed_links:
                for failed_link in failed_links:
                    writer.writerow({'file_name': csv_file_name, 'failed_link': failed_link})
                print(f"Failed links from {csv_file_name} stored in {failed_csv_path}")
            else:
                print(f"All URLs from {csv_file_name} processed successfully.")

csv_folder_path = r'/mnt/HDFS1/language_nlp/malayalam_nlp_shristi/codes/news12/csv_folder'
corpus_directory = r'/mnt/HDFS1/language_nlp/malayalam_nlp_shristi/codes/news12/extracted_text'
failed_csv_path = r'/mnt/HDFS1/language_nlp/malayalam_nlp_shristi/codes/news12/failed_csv/failed1.csv'

process_csv_folder(csv_folder_path, corpus_directory, failed_csv_path)

/mnt/HDFS1/language_nlp/malayalam_nlp_team5/codes/news12/failed_2.csv

In [1]:
# script 5

import requests
from bs4 import BeautifulSoup
import io
import json
import time
from requests import Session

url = 'https://www.madhyamam.com/kerala/'
head = 'https://www.madhyamam.com'
page = 0

while True:
    page += 1
    newsDivss = BeautifulSoup(requests.get(url + str(page)).text, 'html.parser').find_all('div', {'class': 'other-news'})
    for newsDiv in newsDivss:
        newsDivs = newsDiv.find_all('div', {'class': 'exclude-news-item'})
        for linksDiv in  newsDivs:
            headingA = linksDiv.find('div', {'class': 'heading'}).find('a')
            heading = headingA.text
            link = head + headingA.get('href')
            print(page, link)
            newsDatadiv = BeautifulSoup(requests.get(link).text, 'html.parser').find('div', {'class': 'story'})
            if newsDatadiv is None:
                continue
            else:
                newsData = newsDatadiv.findChildren('p')
            with io.open('extracted_text.txt', 'a', encoding='utf-8') as file:
                file.write(heading)
                file.write("\n")
                for data in newsData:
                    file.write(data.text)
                    file.write("\n")

1 https://www.madhyamam.com/kerala/gold-recovered-on-mt-vasudevan-nair-theft-1338563
1 https://www.madhyamam.com/kerala/finally-children-came-to-see-their-father-1338562
1 https://www.madhyamam.com/kerala/rates-are-displayed-private-hospitals-1338561
1 https://www.madhyamam.com/kerala/tp-madhavan-funeral-1338560
1 https://www.madhyamam.com/kerala/wrongfully-detained-the-teacher-filed-a-complaint-against-the-police-1338558
1 https://www.madhyamam.com/kerala/sreenath-bhasi-and-prayaga-martin-were-interrogated-in-the-drug-case-1338557
1 https://www.madhyamam.com/kerala/bribery-case-idukki-dmo-remanded-1338553
1 https://www.madhyamam.com/kerala/citu-workers-obstructed-the-construction-of-the-pavement-1338549
1 https://www.madhyamam.com/kerala/suresh-gopi-says-he-got-call-from-cm-pinarayi-vijayan-regarding-joining-cpim-1338548
1 https://www.madhyamam.com/kerala/pv-anwar-mla-protest-1338547
1 https://www.madhyamam.com/kerala/actor-sreenath-bhasi-was-questioned-in-kochi-in-connection-with-a-d

KeyboardInterrupt: 

In [ ]:
# script 6

import requests
from bs4 import BeautifulSoup
import io
import json
import time
from requests import Session

url = 'https://www.madhyamam.com/kerala/'
head = 'https://www.madhyamam.com'
page = 0

while True:
    page += 1
    newsDivss = BeautifulSoup(requests.get(url + str(page)).text, 'html.parser').find_all('div', {'class': 'other-news'})
    for newsDiv in newsDivss:
        newsDivs = newsDiv.find_all('div', {'class': 'exclude-news-item'})
        for linksDiv in  newsDivs:
            headingA = linksDiv.find('div', {'class': 'heading'}).find('a')
            heading = headingA.text
            link = head + headingA.get('href')
            print(page, link)
            newsDatadiv = BeautifulSoup(requests.get(link).text, 'html.parser').find('div', {'class': 'story'})
            if newsDatadiv is None:
                continue
            else:
                newsData = newsDatadiv.findChildren('p')
            with io.open('extracted_text.txt', 'a', encoding='utf-8') as file:
                file.write(heading)
                file.write("\n")
                for data in newsData:
                    file.write(data.text)
                    file.write("\n")

In [ ]:
# script 7

import requests
from bs4 import BeautifulSoup
import csv
import os
import time

def fetch_links(page_number):
    url = f"https://www.manoramaonline.com/news/latest-news.html?page={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('a', href=True)

    links = []
    for article in articles:
        link = article['href']
        if link.startswith('/'):
            link = f"https://www.manoramaonline.com/news/latest-news.html{link}"
        links.append(link)

    return links

def save_links_to_csv(folder_path, page_number, links):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    filename = os.path.join(folder_path, f"{page_number}.csv")
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Link"])
        for link in links:
            writer.writerow([link])

if _name_ == "_main_":
    folder_path = '/mnt/HDFS1/language_nlp/malayalam_nlp_shristi/codes/news12/csv_folder'
    total_pages = 100
    for page_number in range(1, total_pages + 1):
        try:
            links = fetch_links(page_number)
            save_links_to_csv(folder_path, page_number, links)
            print(f"Page {page_number} processed successfully.")
        except Exception as e:
            print(f"An error occurred on page {page_number}: {e}")
            continue